In [67]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import datasets, linear_model

In [65]:
def read_goog_sp500_data():
    googFile = 'data/GOOG.csv'
    spFile = 'data/GSPC.csv'
    
    goog = pd.read_csv(googFile, sep=",", usecols=[0,6], names=['Date', 'Goog'], header=0)
    sp = pd.read_csv(spFile, sep=",", usecols=[0,6], names=['Date', 'SP500'], header=0)
    goog['SP500'] = sp['SP500']
    
    # Format the date date object into a datetime
    goog['Date'] = pd.to_datetime(goog['Date'], format='%Y/%m/%d')
    
    returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64', 'int64']]].pct_change()
    xData = np.array(returns['SP500'][1:])
    yData = np.array(returns['Goog'][1:])
    return xData, yData

In [66]:
xData, yData = read_goog_sp500_data()
googModel = linear_model.LinearRegression()
googModel.fit(xData.reshape(-1, 1), yData.reshape(-1, 1))
print(googModel.coef_)
print(googModel.intercept_)

[[1.03551531]]
[0.01556539]


In [83]:
W = tf.Variable(tf.zeros([1,1]))
b = tf.Variable(tf.zeros([1]))

x = tf.placeholder(tf.float32, [None, 1])

Wx = tf.matmul(x, W)

y = Wx + b

y_ = tf.placeholder(tf.float32, [None, 1])
cost = tf.reduce_mean(tf.square(y_ -y))

train_step_constant = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

def trainWithOnePointPerEpoch(steps, train_step):
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for i in range(steps):
            xs = np.array([[xData[i % len(yData)]]])
            ys = np.array([[yData[i % len(yData)]]])
            
            feed = {x: xs, y_: ys}
            
            sess.run(train_step, feed_dict=feed)
            
            if (i + 1) % 1000 == 0:
                print("After %d iteration:" % i)
                print("W: %f" % sess.run(W))
                print("b: %f" % sess.run(b))
                
                print("cost: %f" % sess.run(cost, feed_dict=feed))
                

trainWithOnePointPerEpoch(1000, train_step_constant)

After 999 iteration:
W: 0.986041
b: -0.034240
cost: 0.000006
